# AIML Coursework marker

<div class="alert alert-block alert-danger"> <b>REMEMBER:</b> You need to make sure you are running this code within the virtual environment you created using 'AIenv'.<br> In Jupyter click on the kernel menu then change-kernel. In VSCode use the kernel selector in the top-right hand corner </div>

In [1]:
import re
import aiml
import random
import time
time.clock = time.time

def preprocessSingleInput(bot,theInput):
    # run the input through the 'normal' subber- only wortks for a single sentence
    subbed1 = bot._subbers['normal'].sub(theInput).upper()
    subbed2 = re.sub(bot._brain._puncStripRE, " ", subbed1)
    return(subbed2)
    

# Next cell sets up variables
- You can change the amount of debugging information printed to screen by setting debug=True
- you can change the name of your input file to something other than "student.aiml" if you want.
- **Dont change anything else**

In [2]:
debug = True
debug2 = True
theAIMLfile = 'chatbot_evan.aiml'
theQuestionsFileName = "coursework-questions-and-responses.txt"
responsesFileName = theAIMLfile[:-5] +"-responses.txt"
feedbackFileName = theAIMLfile[:-5] +"-feedback.txt"
NUMQS =45
NUMCONTEXTQS=3
contextQuestions = [35,42,44]

# Read the questions and answer from file, then randomise the order

In [3]:
#declare arrays to hold the questions and answers
questions = []
responses = []

# read the questions and answers in
# Using readline() 
qFile = open(theQuestionsFileName, 'r') 
thisQ = 0
  
while True: 
    # Get next line from file 
    line = qFile.readline() 
    if not line: 
        print("unexpected end of file")
        break
    # should be a question
    elif (line[0] != 'Q' ):
        print("didn't get expected question marker Q")
        break
    elif ( int(line[1:3]) != thisQ):
        print("question had wrong number")
        break
    else:
        questions.append( line[5:-1])
        if(debug2):
            print("question {} is: {}".format(thisQ,questions[thisQ]))        
        
    line = qFile.readline() # next line should be the corresponding answer
    if not line: 
        print("unexpected end of file")
        break
    elif (line[0] != 'A' ):
        print("didn't get expected answer marker A")
        break
    elif ( int(line[1:3]) != thisQ):
        print("answer had wrong number")
        break
    else:
        responses.append(line[5:-1])
        if(debug2):
            print("response {} is: {}".format(thisQ,responses[thisQ]))
    
    thisQ += 1
    # then read the empty line separating QnA paits
    line = qFile.readline()
    
    # if line is empty 
    # end of file is reached 
    if not line: 
        break
    if(debug2):
        print("")

qFile.close() 




# shuffle the order of the questions except the **three** context-dependent ones
CQ1 = contextQuestions[0]
CQ2 = contextQuestions[1]
CQ3 = contextQuestions[2]
toremove= [(CQ1 - 1),CQ1,(CQ2 - 1),CQ2,(CQ3 - 1),CQ3]
#print(toremove)
# make a shuffled list with the numbers 1...NUMQs except the ones above in
order = []
for i in range (NUMQS):
    if i not in toremove:
        order.append(i)
random.shuffle(order)

#put the context dependent Qs and precursors back in
order.insert(10,(CQ1 -1))
order.insert(11,CQ1)
order.insert(20,(CQ2-1))
order.insert(21,CQ2)
order.insert(30, (CQ3-1))
order.insert(31,CQ3)
#print(order)
#print ( len(order))
for i in range (NUMQS):
    if i not in order:
        print("{} is missing".format(i))

# check that there are the right number
if (thisQ <NUMQS ):
    print("error, only {} question-answer pairs read".format(thisQ))
elif (len(questions) < NUMQS or len(responses)<NUMQS):
    print("error, somehow the questions or responses have not all be saved")
    if(debug):
        print(" {} questions and {}responses read, thisQ = {}".format(len(questions),len(responses),thisQ))
else: 
    print('{} question-response pairs read for testing your bot'.format(thisQ))

question 0 is: What is long-term knowledge?
response 0 is: Long-term knowledge is valid over a period of time.

question 1 is: What is short-term knowledge?
response 1 is: Short-term knowledge is only valid for a particular situation or run of a programme.

question 2 is: What is generic knowledge?
response 2 is: Generic knowledge is relevant to more than one application or domain.

question 3 is: What is domain-specific knowledge?
response 3 is: Domain-specific knowledge is only applicable to one specific application or domain.

question 4 is: What is a fact?
response 4 is: A fact is a statement which asserts that a relationship holds for an object-subject pair.

question 5 is: What is a rule?
response 5 is: A rule is a statement that generates new knowledge by asserting that if one fact is true, then another is also true.

question 6 is: Give me an example of long-term generic knowledge.
response 6 is: The formal specification of the python programming language.

question 7 is: Give 

# Create the chatbot

In [4]:
# Create Chatbot and read the candidate AIML file
checkBot = aiml.Kernel()
checkBot.verbose(True)

## Clear any old categories,  reload the AIML file

In [5]:
checkBot.resetBrain()
checkBot.learn(theAIMLfile)

# How many categories were correctly read
numCategories = checkBot.numCategories()
print( "After reading your file the bot has {} categories".format(numCategories))
print( "Remember that the bot will overwrite categories with the same pattern, that and topic".format(numCategories))
print('This number should help you fix misformed categories if needed\n')


Loading chatbot_evan.aiml...done (0.00 seconds)
After reading your file the bot has 5 categories
Remember that the bot will overwrite categories with the same pattern, that and topic
This number should help you fix misformed categories if needed



### See how frequently different language constrcuts have been used

In [6]:
# either figure out how to query the bot categories
## or open the student.aiml file and read it line by line looking for <srai> <set> <star/> and <that>
file2 = open(theAIMLfile,'r')
srai_count = 0
set_count = 0
wildcard_count=0
starslash_count=0
that_count = 0
condition_count= 0

#read through line by line coutning uise of AIML constructs
while(True):
    line = file2.readline()
    if not line:
        break
    if "<srai>" in line:
        srai_count += 1
    if "<set" in line: # just use start - they ar hopefullty defining a name for their variable
        set_count += 1
    if ("*" in line) or ("_" in line) or ("^" in line) or ("#" in line):
        wildcard_count +=1
    if "<star" in line: #just look for start of tag in case they used indexing
        starslash_count += 1
    if "<that" in line: #just look for start of tag in case they used indexing
        that_count +=1
    if "<that" in line: #just look for start of tag in case they used indexing
        condition_count +=1
file2.close()       
        


# Ask the questions, check and store the responses

In [7]:


# initialise score
numCorrect = 0
numContextQsCorrect=0
numNoMatch=0
responsesFile = open(responsesFileName,'w')

for q in range (NUMQS):
    thisQ = order[q]
    #get bot's response to question
    botResponse = checkBot.respond(questions[thisQ])
    if(botResponse==""):
        numNoMatch +=1
    responsesFile.write('Q{:2d}: {}\n'.format(thisQ, questions[thisQ]))
    responsesFile.write('Expected response: {}\n'.format(responses[thisQ]))
    responsesFile.write('Your bot response: {}\n'.format(botResponse))
    # check if it matches the required input
    if botResponse == responses[thisQ] :
        #print('question {} answered correctly'.format(thisQ))
        responsesFile.write('*** Question answered correctly\n\n')
        numCorrect +=1
        if thisQ in contextQuestions:
            numContextQsCorrect +=1
    else:
        responsesFile.write('Question answered incorrectly\n\n')
        if(debug):
            theInput = questions[thisQ]
            print('Q{} {}\n gets preprocessed as:{}'.format(thisQ,theInput,preprocessSingleInput(checkBot,theInput)))
            print(' expected :' +responses[thisQ])
            print(' got      :' +botResponse)
            lastThat = checkBot.getPredicate("_outputHistory")

# write final line to log file and exit
responsesFile.write(' In total you got {} questions correct'.format(numCorrect))
responsesFile.close()

Q36 Please provide an example of a rule.
 gets preprocessed as:PLEASE PROVIDE AN EXAMPLE OF A RULE 
 expected :If isAlive(X) then NOT isDead(X).
 got      :
Q39 Please myBot what is a rule?
 gets preprocessed as:PLEASE MYBOT WHAT IS A RULE 
 expected :A rule is a statement that generates new knowledge by asserting that if one fact is true, then another is also true.
 got      :
Q28 Illustrate the concept of a fact.
 gets preprocessed as:ILLUSTRATE THE CONCEPT OF A FACT 
 expected :All coursework for this module is run through automatic and effective plagiarism checks.
 got      :
Q29 Illustrate the concept of a rule.
 gets preprocessed as:ILLUSTRATE THE CONCEPT OF A RULE 
 expected :If isAlive(X) then NOT isDead(X).
 got      :
Q22 Tell me about a fact.
 gets preprocessed as:TELL ME ABOUT A FACT 
 expected :A fact is a statement which asserts that a relationship holds for an object-subject pair.
 got      :
Q9 Give me an example of short-term domain-specific knowledge.
 gets preprocess

# Calculate final score and feedback

In [8]:

feedbackFile = open(feedbackFileName,'w')


# calculate final score
finalScore= numCorrect 
# if all questions correct then we start rewarding go solutions
if (numCorrect==NUMQS):
    if (numCategories <10):
        finalScore = 100
    else:
        finalScore = 90 - numCategories

# provide output for DEWIS
feedbackFile.write('<SCORE>{}</SCORE>\n'.format(finalScore))

fstart=  "<MESSAGE>"
fend = "</MESSAGE>\n"

feedback = fstart + "After removing duplicates, your bot used " + str(numCategories) + " categories" +fend
feedbackFile.write(feedback)

# what did the submission get wrong and why?
if(numCorrect< NUMQS):
    feedback = fstart+ "Your bot answered one or more questions incorrectly." +fend 
    feedbackFile.write(feedback)
    feedback = fstart + "File " + responsesFileName + " has more details of your bots responses." +fend
    feedbackFile.write(feedback)
    feedback = fstart + "Common mistakes are typos or extra spaces" +fend
    feedbackFile.write(feedback)
    
    if(numNoMatch>0):
        feedback = fstart + "For " + str(numNoMatch) +" questions your bot did not have a matching category." +fend
        feedbackFile.write(feedback)
    contextErrors = NUMCONTEXTQS - numContextQsCorrect
    if( contextErrors >0 ):
        feedback= fstart +"Your bot answered incorrectly for " + str(contextErrors) + " questions that require a sense of context." +fend
        feedbackFile.write(feedback)

else: #
    feedback = fstart +"Your bot answered every question correctly using " + str(numCategories) + " categories" +fend
    feedbackFile.write(feedback)
    if ( srai_count==0  or wildcard_count ==0 or starslash_count==0):
        feedback = fstart+ "You can improve your score by generalising using srai and wildcards." + fend
        feedbackFile.write(feedback)
    if (set_count==0 or that_count==0):
        feedback = fstart + "You can improve your score by remembering context and what the conversation is talking about." +fend
        feedbackFile.write(feedback)
    if(condition_count==0):
        feedback = fstart + "You can use <condition> to change behaviour within a category." +fend
        feedbackFile.write(feedback)
    if(numCategories <=11):
        feedback = fstart + "Congratulations, you have matched Jim's score!" +fend
        feedbackFile.write(feedback)


feedbackFile.close()

# Uncomment the cell below if you want to run your bot interactively

In [9]:
#keepgoing= True
#while(keepgoing):
#    nextInput = input("Enter your message >> ")
#    if (nextInput=='bye'):
#       keepgoing= False
#    else:
#        print (checkBot.respond(nextInput))